In [1]:
# Install llama.cpp
!git clone https://github.com/ggerganov/llama.cpp
!cd llama.cpp && git pull && make clean && LLAMA_CUBLAS=1 make
!pip install -r llama.cpp/requirements.txt

Cloning into 'llama.cpp'...
remote: Enumerating objects: 17969, done.
remote: Counting objects: 100% (7567/7567), done.
remote: Compressing objects: 100% (473/473), done.
remote: Total 17969 (delta 7341), reused 7154 (delta 7093), pack-reused 10402
Receiving objects: 100% (17969/17969), 20.79 MiB | 21.48 MiB/s, done.
Resolving deltas: 100% (12581/12581), done.
Already up to date.
I ccache not found. Consider installing it for faster compilation.
I llama.cpp build info: 
I UNAME_S:   Linux
I UNAME_P:   x86_64
I UNAME_M:   x86_64
I CFLAGS:    -I. -Icommon -D_XOPEN_SOURCE=600 -D_GNU_SOURCE -DNDEBUG  -std=c11   -fPIC -O3 -Wall -Wextra -Wpedantic -Wcast-qual -Wno-unused-function -Wshadow -Wstrict-prototypes -Wpointer-arith -Wmissing-prototypes -Werror=implicit-int -Werror=implicit-function-declaration -pthread -march=native -mtune=native -Wdouble-promotion 
I CXXFLAGS:  -I. -Icommon -D_XOPEN_SOURCE=600 -D_GNU_SOURCE -DNDEBUG  -std=c++11 -fPIC -O3 -Wall -Wextra -Wpedantic -Wcast-qual -Wno-un

In [2]:
# Download model
!git lfs install
!git clone https://huggingface.co/Siddharthvij10/MistralSharded2

Git LFS initialized.
Cloning into 'MistralSharded2'...
remote: Enumerating objects: 27, done.
remote: Counting objects: 100% (24/24), done.
remote: Compressing objects: 100% (23/23), done.
remote: Total 27 (delta 3), reused 0 (delta 0), pack-reused 3
Unpacking objects: 100% (27/27), 463.65 KiB | 2.38 MiB/s, done.
Filtering content: 100% (9/9), 13.48 GiB | 55.64 MiB/s, done.


In [3]:
# Convert weights to fp16
!python llama.cpp/convert.py MistralSharded2 --outtype f16 --outfile "MistralSharded2/mistralsharded2.fp16.bin"

Loading model file MistralSharded2/pytorch_model-00001-of-00008.bin
Loading model file MistralSharded2/pytorch_model-00001-of-00008.bin
Loading model file MistralSharded2/pytorch_model-00002-of-00008.bin
Loading model file MistralSharded2/pytorch_model-00003-of-00008.bin
Loading model file MistralSharded2/pytorch_model-00004-of-00008.bin
Loading model file MistralSharded2/pytorch_model-00005-of-00008.bin
Loading model file MistralSharded2/pytorch_model-00006-of-00008.bin
Loading model file MistralSharded2/pytorch_model-00007-of-00008.bin
Loading model file MistralSharded2/pytorch_model-00008-of-00008.bin
params = Params(n_vocab=32000, n_embd=4096, n_layer=32, n_ctx=32768, n_ff=14336, n_head=32, n_head_kv=8, n_experts=None, n_experts_used=None, f_norm_eps=1e-05, rope_scaling_type=None, f_rope_freq_base=10000.0, f_rope_scale=None, n_orig_ctx=None, rope_finetuned=None, ftype=<GGMLFileType.MostlyF16: 1>, path_model=PosixPath('MistralSharded2'))
Found vocab files: {'tokenizer.model': PosixP

In [4]:
# Quantization - Requires GPU RAM as a mandate. However, does not use much of it.

# As per info on https://huggingface.co/TheBloke/Llama-2-13B-chat-GGML, q4_k_m uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q4_K
!./llama.cpp/quantize "MistralSharded2/mistralsharded2.fp16.bin" "MistralSharded2/mistralsharded2.Q4_K_M.gguf" q4_k_m

ggml_init_cublas: GGML_CUDA_FORCE_MMQ:   no
ggml_init_cublas: CUDA_USE_TENSOR_CORES: yes
ggml_init_cublas: found 1 CUDA devices:
  Device 0: Tesla T4, compute capability 7.5, VMM: yes
main: build = 2112 (4b7b38be)
main: built with cc (Ubuntu 11.4.0-1ubuntu1~22.04) 11.4.0 for x86_64-linux-gnu
main: quantizing 'MistralSharded2/mistralsharded2.fp16.bin' to 'MistralSharded2/mistralsharded2.Q4_K_M.gguf' as Q4_K_M
llama_model_loader: loaded meta data with 19 key-value pairs and 291 tensors from MistralSharded2/mistralsharded2.fp16.bin (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = .
llama_model_loader: - kv   2:                       llama.context_length u32              = 32768
llama_model_loader: - kv   3:                     

In [5]:
import os
# There are 32 layers in mistral 7B. Hence offloading all 32 layers to GPU while loading for inference below.
!./llama.cpp/main -m "MistralSharded2/mistralsharded2.Q4_K_M.gguf" -n 35 --color -ngl 0 -p "Automated machine learning"

Log start
main: build = 2112 (4b7b38be)
main: built with cc (Ubuntu 11.4.0-1ubuntu1~22.04) 11.4.0 for x86_64-linux-gnu
main: seed  = 1707551689
ggml_init_cublas: GGML_CUDA_FORCE_MMQ:   no
ggml_init_cublas: CUDA_USE_TENSOR_CORES: yes
ggml_init_cublas: found 1 CUDA devices:
  Device 0: Tesla T4, compute capability 7.5, VMM: yes
llama_model_loader: loaded meta data with 20 key-value pairs and 291 tensors from MistralSharded2/mistralsharded2.Q4_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = .
llama_model_loader: - kv   2:                       llama.context_length u32              = 32768
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:     